# Convert Json Schema ver 03 captured data to Sqlite3
Still a bit trail and error, but the Json format is fixed enough to start working on full file conversion and upload into the Sqlite database.

In [1]:
from datetime import datetime
import json
import sqlite3
import pandas as pd

In [2]:
#sqlSchemaInitialisation = open('RawData/developmentSchema03.sql', encoding='utf8').read()
##print(sqlSchemaInitialisation)
#cnx = sqlite3.connect(":memory:")
#cur = cnx.cursor()
#print(cnx)
#print(cur)
#cur.executescript(sqlSchemaInitialisation)

In [3]:
def getKnownLeaders(cnx):
    cur = cnx.cursor()
    sqlSelect = 'SELECT leaderId, name, civ FROM leader'
    return pd.read_sql_query(sqlSelect, cnx)

def getLeaderId(name, civ, knownLeaders):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(knownLeaders[(knownLeaders.name == name) & (knownLeaders.civ == civ)].iloc[0].leaderId)
    except IndexError:
        return None

In [4]:
# Without the encoding the json gets mangled for non-ASCII characters
jsonData = open('RawData/20181129_01.json', encoding='utf8').read()

In [5]:
rawData = json.loads(jsonData)

In [6]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

## Populate the game table

In [7]:
# Python creates a list by default, we know we only have 1 so select first element only!
gameData = rawData['gameConfig'][0]
print(gameData)
print(type(gameData))

{'startEra': 'Ancient Era', 'mapSize': 'Small', 'leaderCiv': 'Aztec', 'leaderName': 'Montezuma', 'ruleSet': 'Expansion: Rise and Fall', 'date': '20181129', 'leaders': [{'leaderCiv': 'Aztec', 'leaderName': 'Montezuma', 'isMajor': True, 'isHuman': True}, {'leaderCiv': 'Indonesia', 'leaderName': 'Gitarja', 'isMajor': True, 'isHuman': False}, {'leaderCiv': 'Netherlands', 'leaderName': 'Wilhelmina', 'isMajor': True, 'isHuman': False}, {'leaderCiv': 'Rome', 'leaderName': 'Trajan', 'isMajor': True, 'isHuman': False}, {'leaderCiv': 'Arabia', 'leaderName': 'Saladin', 'isMajor': True, 'isHuman': False}, {'leaderCiv': 'Mongolia', 'leaderName': 'Genghis Khan', 'isMajor': True, 'isHuman': False}, {'leaderCiv': 'Antananarivo', 'leaderName': 'Antananarivo', 'isMajor': False, 'isHuman': False}, {'leaderCiv': 'Valletta', 'leaderName': 'Valletta', 'isMajor': False, 'isHuman': False}, {'leaderCiv': 'Muscat', 'leaderName': 'Muscat', 'isMajor': False, 'isHuman': False}, {'leaderCiv': 'Hong Kong', 'leaderNa

In [8]:
#dateString = rawData['date']
#date = datetime.strptime(dateString, '%Y%m%d').date()
game = {
    'gameId': None,
    'date': datetime.strptime(gameData['date'], '%Y%m%d').date(),
    'difficulty': gameData['difficulty'],
    'mapType': 'Continents',
    'mapSize': gameData['mapSize'],
    'gameSpeed': gameData['gameSpeed'],
    'startEra': gameData['startEra'],
    'ruleSet': gameData['ruleSet'],
}
print(game)

{'gameId': None, 'date': datetime.date(2018, 11, 29), 'difficulty': 'Prince', 'mapType': 'Continents', 'mapSize': 'Small', 'gameSpeed': 'Standard', 'startEra': 'Ancient Era', 'ruleSet': 'Expansion: Rise and Fall'}


In [9]:
cur.execute('INSERT INTO game VALUES(:gameId, :date, :difficulty, :mapType, :mapSize, :gameSpeed, :startEra, :ruleSet)',
    game)
gameId = cur.lastrowid
print("gameID: ", gameId)

gameID:  8


In [10]:
cnx.commit()

## Add "new" leaders to the leader table.
The leader table is a list of all the leaders we've captured data for, regardless of when they were first seen. We record the relationship between a specific game/instance with the leaders in the gameLeader table.

In [11]:
knownLeaders = getKnownLeaders(cnx)
print(knownLeaders)

    leaderId                  name            civ
0          7             Amanitore          Nubia
1         54          Antananarivo   Antananarivo
2         35               Antioch        Antioch
3         14                Armagh         Armagh
4         28              Auckland       Auckland
5          4               Babylon        Babylon
6          3         Bandar Brunei  Bandar Brunei
7         13              Brussels       Brussels
8         47          Buenos Aires   Buenos Aires
9         48              Carthage       Carthage
10        11   Catherine de Medici         France
11        22             Cleopatra          Egypt
12        26                 Cyrus         Persia
13        51  Frederick Barbarossa        Germany
14        12                Geneva         Geneva
15        21          Genghis Khan       Mongolia
16         1             Gilgamesh        Sumeria
17        20               Gitarja      Indonesia
18        45               Granada        Granada


In [12]:
result = []
for leader in gameData['leaders']:
    name = leader['leaderName']
    civ = leader['leaderCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    if leaderId is None:
        newLeader = {
            'leaderId': None,
            'name': name,
            'civ': civ,
            'isMajor': leader['isMajor']
        }
        result.append(newLeader)

for row in result:
    print(row)

In [13]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO leader VALUES(:leaderId, :name, :civ, :isMajor)', row)

In [14]:
cnx.commit()

# Populate gameLeader table

In [15]:
#knownLeaders = getKnownLeaders(cnx)
#print(knownLeaders)
#humanLeaderName = gameData['leader']
#print(humanLeaderName)
#print(gameId)

In [16]:
# Update knownLeaders to get the newly added one(s)
knownLeaders = getKnownLeaders(cnx)
result = []
for leader in gameData['leaders']:
    name = leader['leaderName']
    civ = leader['leaderCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    newGameLeader = {
        'gameId': gameId,
        'leaderId':leaderId,
        'isHuman':leader['isHuman'],
    }
    result.append(newGameLeader)
    
for row in result:
    print(row)

{'gameId': 8, 'leaderId': 2, 'isHuman': True}
{'gameId': 8, 'leaderId': 20, 'isHuman': False}
{'gameId': 8, 'leaderId': 8, 'isHuman': False}
{'gameId': 8, 'leaderId': 25, 'isHuman': False}
{'gameId': 8, 'leaderId': 50, 'isHuman': False}
{'gameId': 8, 'leaderId': 21, 'isHuman': False}
{'gameId': 8, 'leaderId': 54, 'isHuman': False}
{'gameId': 8, 'leaderId': 46, 'isHuman': False}
{'gameId': 8, 'leaderId': 33, 'isHuman': False}
{'gameId': 8, 'leaderId': 5, 'isHuman': False}
{'gameId': 8, 'leaderId': 36, 'isHuman': False}
{'gameId': 8, 'leaderId': 30, 'isHuman': False}
{'gameId': 8, 'leaderId': 13, 'isHuman': False}
{'gameId': 8, 'leaderId': 34, 'isHuman': False}
{'gameId': 8, 'leaderId': 3, 'isHuman': False}


In [17]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO gameLeader VALUES(:gameId, :leaderId, :isHuman)', row)

In [18]:
cnx.commit()

# Populate citySettled and gameCity Tables

In [19]:
#knownLeaders = getKnownLeaders(cnx)
citySettledLog = rawData['citySettledLog']
#print(citySettledLog)
for city in citySettledLog:
    settledByName = city['ownerName']
    settledByCiv = city['ownerCiv']
    # Get the leaderId of the player settling the city
    settledById = getLeaderId(settledByName, settledByCiv, knownLeaders)
    cityName = city['cityName']

    sqlInsert = "INSERT INTO citySettledLog VALUES(:cityId, :name, :settledById, :onTurn)"
    parameters = {'cityId': None, 'name': cityName, 'settledById':settledById, 'onTurn':city['turn']}
    cur.execute(sqlInsert, parameters)
    cityId = cur.lastrowid
    print(cityId, cityName, settledById, settledByName,  settledByCiv)
    
    sqlInsert = "INSERT INTO gameCity VALUES(:gameId, :cityId)"
    parameters = {'gameId':gameId, 'cityId':cityId}
    cur.execute(sqlInsert, parameters)

114 Tenochtitlan 2 Montezuma Aztec
115 Cairo 50 Saladin Arabia
116 Antananarivo 54 Antananarivo Antananarivo
117 Valletta 46 Valletta Valletta
118 Muscat 33 Muscat Muscat
119 Hong Kong 5 Hong Kong Hong Kong
120 Hattusa 36 Hattusa Hattusa
121 Jerusalem 30 Jerusalem Jerusalem
122 Brussels 13 Brussels Brussels
123 Kandy 34 Kandy Kandy
124 Bandar Brunei 3 Bandar Brunei Bandar Brunei
125 Majapahit 20 Gitarja Indonesia
126 Amsterdam 8 Wilhelmina Netherlands
127 Qaraqorum 21 Genghis Khan Mongolia
128 Rome 25 Trajan Rome


In [20]:
#cnx.rollback()

In [21]:
cnx.commit()

## Populate cityPlotsSettled

In [22]:
sqlSelect = 'SELECT * FROM gameCityLeaderSettledView WHERE gameId = :gameId'
gameCLS = pd.read_sql_query(sqlSelect, cnx, params={'gameId':gameId})
print(gameCLS)

    gameId  cityId       cityName     leaderName  leaderId      leaderCiv  \
0        8     114   Tenochtitlan      Montezuma         2          Aztec   
1        8     115          Cairo        Saladin        50         Arabia   
2        8     116   Antananarivo   Antananarivo        54   Antananarivo   
3        8     117       Valletta       Valletta        46       Valletta   
4        8     118         Muscat         Muscat        33         Muscat   
5        8     119      Hong Kong      Hong Kong         5      Hong Kong   
6        8     120        Hattusa        Hattusa        36        Hattusa   
7        8     121      Jerusalem      Jerusalem        30      Jerusalem   
8        8     122       Brussels       Brussels        13       Brussels   
9        8     123          Kandy          Kandy        34          Kandy   
10       8     124  Bandar Brunei  Bandar Brunei         3  Bandar Brunei   
11       8     125      Majapahit        Gitarja        20      Indonesia   

In [23]:
# Testing, can safely ignore this!
#ownerCityId = int(gameCLS[gameCLS.cityName == 'Amsterdam'].iloc[0].cityId)
#print(ownerCityId)
#print(type(ownerCityId))

In [24]:
# building up the cityPlotsSettled data for entry...
result = []
for city in rawData['citySettledLog']:
    for plot in city['plots']:
        try:
            ownerCityId = int(gameCLS[gameCLS.cityName == plot['ownerCity']].iloc[0].cityId)
        except IndexError:
            ownerCityId = None
        plotInfo = {
            'plotId': None,
            'ownerCityId': ownerCityId,
            'ring': plot['r'],
            'terrain': plot['terrain'],
            'feature': plot['feature'],
            'resource': plot['resource'],
            'resourceCount': plot['resourceCount'],
            'resourceType': plot['resourceType'],
            'workers': plot['workers'],
            'district': plot['district'],
            'hasRiver': plot['hasRiver'],
            'isWater': plot['isWater'],
            'isLake': plot['isLake'],
            'isCity': plot['isCity'],
        }
        result.append(plotInfo)

print(result[0])
print(result[1])
print(result[2])

{'plotId': None, 'ownerCityId': 114, 'ring': 0, 'terrain': 'Plains', 'feature': 'None', 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'workers': 1, 'district': 'City Center', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': True}
{'plotId': None, 'ownerCityId': 114, 'ring': 1, 'terrain': 'Coast and Lake', 'feature': 'None', 'resource': 'Fish', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': False, 'isWater': True, 'isLake': False, 'isCity': False}
{'plotId': None, 'ownerCityId': 114, 'ring': 1, 'terrain': 'Coast and Lake', 'feature': 'None', 'resource': 'Crabs', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': False, 'isWater': True, 'isLake': False, 'isCity': False}


In [25]:
for row in result:
    cur.execute('INSERT INTO cityPlotsSettled VALUES(:plotId, :ownerCityId, :ring, :terrain, :feature, :resource, :resourceCount, :resourceType, :workers, :district, :hasRiver, :isWater, :isLake, :isCity )',
                row)

In [26]:
#cnx.rollback()

In [27]:
cnx.commit()

## Populate cityPerTurnLog

In [28]:
def getCityId(cityName):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[gameCLS.cityName == cityName].iloc[0].cityId)
    except IndexError:
        return None
    
def getLeaderId(ownerName, ownerCiv):
    try:
        # The iloc is necessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[(gameCLS.leaderName  == ownerName) & (gameCLS.leaderCiv == ownerCiv)].iloc[0].leaderId)
    except IndexError:
        return None

In [29]:
# building up the cityPerTurnLog data for entry...
result = []
for logEntry in rawData['cityPerTurnLog']:
    turnInfo = {
        'cityId': getCityId(logEntry['cityName']),
        'turn': int(logEntry['turn']),
        'ownerId': getLeaderId(logEntry['ownerName'], logEntry['ownerCiv']),
        'foodPerTurn': logEntry['foodPerTurn'],
        'foodToolTip': logEntry['foodToolTip'],
        'productionPerTurn': logEntry['productionPerTurn'],
        'productionToolTip': logEntry['productionToolTip'],
        'goldPerTurn': logEntry['goldPerTurn'],
        'goldToolTip': logEntry['goldToolTip'],
        'sciencePerTurn': logEntry['sciencePerTurn'],
        'scienceToolTip': logEntry['scienceToolTip'],
        'culturePerTurn': logEntry['culturePerTurn'],
        'cultureToolTip': logEntry['cultureToolTip'],
        'faithPerTurn': logEntry['faithPerTurn'],
        'faithToolTip': logEntry['faithToolTip'],
        'population': logEntry['population'],
        'housing': logEntry['housing'],
        'amenities': logEntry['amenities'],
        'amenitiesNeeded': logEntry['amenitiesNeeded'],
        'happiness': logEntry['happiness'],
    }
#    print(logEntry['ownerName'], logEntry['ownerCiv'], getLeaderId(logEntry['ownerName'], logEntry['ownerCiv']))
    result.append(turnInfo)

print(result[0])
print(result[1])
print(result[-1])

{'cityId': 114, 'turn': 1, 'ownerId': 2, 'foodPerTurn': '3.00', 'foodToolTip': '+3 from Worked Tiles', 'productionPerTurn': '5.25', 'productionToolTip': '+3 from Worked Tiles[NEWLINE]+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+5pct (+0.2) from Amenities', 'goldPerTurn': '5.25', 'goldToolTip': '+5 from Buildings[NEWLINE]   +5 from Palace[NEWLINE]+5pct (+0.2) from Amenities', 'sciencePerTurn': '2.63', 'scienceToolTip': '+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+0.5 from Population[NEWLINE]+5pct (+0.1) from Amenities', 'culturePerTurn': '1.36', 'cultureToolTip': '+1 from Buildings[NEWLINE]   +1 from Palace[NEWLINE]+0.2 from Population[NEWLINE]+5pct (+0) from Amenities', 'faithPerTurn': '0.00', 'faithToolTip': '+5pct (+0) from Amenities', 'population': 1, 'housing': 6, 'amenities': 1, 'amenitiesNeeded': 0, 'happiness': 'Happy'}
{'cityId': 115, 'turn': 1, 'ownerId': 50, 'foodPerTurn': '4.00', 'foodToolTip': '+4 from Worked Tiles', 'productionPerTurn': '6.25', 'production

In [30]:
for row in result:
    cur.execute('INSERT INTO cityPerTurnLog VALUES(:cityId, :turn, :ownerId, :foodPerTurn, :foodToolTip, :productionPerTurn, :productionToolTip, :goldPerTurn, :goldToolTip, :sciencePerTurn, :scienceToolTip, :culturePerTurn, :cultureToolTip, :faithPerTurn, :faithToolTip, :population, :housing, :amenities, :amenitiesNeeded, :happiness)',
                row)

In [31]:
cnx.rollback()

In [32]:
cnx.commit()

In [33]:
cnx.close()